In [ ]:
import torch
import qtorch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter

### Hyperparameters

In [ ]:
batch_size = 100
hidden_size = 128
learning_rate = 0.1
epochs = 50
momentum = 0.9
rounding = "stochastic"


### Common Variable

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_name = "./test_grafici/fp14_2_" + rounding + "_1000_mom09__TEST"

### Load Dataset

In [ ]:
training_data = datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)

training_loader = torch.utils.data.DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=len(test_data), shuffle=False)

# Images have same size
input_size = np.prod(training_data[0][0].shape[1:])
output_size = 10

# images, labels = training_loader.next()

# plt.imshow(images[1][0], cmap="gray")
# plt.show()


### Quantization setting

In [ ]:
from qtorch.quant import Quantizer, fixed_point_quantize, float_quantize
from qtorch.optim import OptimLP

wl = 14
fl = 2

forward_num = qtorch.FixedPoint(wl=wl, fl=fl)
backward_num = qtorch.FixedPoint(wl=wl, fl=fl)

# forward_num = qtorch.FloatingPoint(exp=5, man=10)
# backward_num = qtorch.FloatingPoint(exp=5, man=10)

Q = Quantizer(forward_number=forward_num, backward_number=backward_num, forward_rounding=rounding, backward_rounding=rounding)
# Q = lambda x : x

weight_quant = lambda x : fixed_point_quantize(x, wl=wl,fl=fl, rounding=rounding)
gradient_quant = lambda x : fixed_point_quantize(x, wl=wl,fl=fl, rounding=rounding)
acc_quant = lambda x : fixed_point_quantize(x, wl=2*wl,fl=2*fl, rounding=rounding)

### Neural Network

In [ ]:
class LinearModule(nn.Module):
    
    def __init__(self, input_size, output_size, hidden_size):
        super(LinearModule, self).__init__()

        self.l1 = nn.Linear(input_size, hidden_size)
        self.act1 = nn.ReLU()

        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.act2 = nn.ReLU()

        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = Q(self.l1(x))
        out = Q(self.act1(out))
        out = Q(self.l2(out))
        out = Q(self.act2(out))
        out = Q(self.output_layer(out))
        return out

model = LinearModule(input_size, output_size, hidden_size)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=momentum, lr=learning_rate)
optimizer = OptimLP(optimizer,weight_quant=weight_quant, grad_quant=gradient_quant, acc_quant=acc_quant)

### Model Test

In [ ]:
def getAccuracy(model):

    with torch.no_grad():
        images, labels = iter(test_loader).next()
        images = images.to(device)
        labels = labels.to(device)
        output = model(images.reshape(-1, input_size))
        _, predictions = torch.max(output, 1)
        acc = (predictions == labels).sum().item()
        acc = 100 * acc / len(images)

        return acc

### Training

In [ ]:
writer = SummaryWriter(file_name)

for epoch in range(epochs):

    loss = None

    for i, (images, labels) in enumerate(training_loader):
        
        step = (i+1) + epoch * len(training_loader)

        # Forward phase
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        predictions = model(images)
        loss = loss_function(predictions, labels)

        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"epoch = {epoch+1}/{epochs}, step = {i}/{len(training_loader)}, loss = {loss}")
            
    # Save on tensorboard
    writer.add_scalar("Training loss", loss, epoch)

    # Evaluate accuracy on test data
    acc = getAccuracy(model)
    writer.add_scalar("Validation accuracy", acc, epoch)

    # Update learning rate
    for g in optimizer.param_groups:
        g['lr'] = learning_rate * 0.95

writer.close()

## Tipologie di test
- float32
- float16
- fixed point
    - 14(2)
        - nearest
        - stochastic
    - 12(4)
        - nearest
        - stochastic